# Recession Forecaster

The United States as of today (October, 2019) has experienced the longest expansion cycle, yet the there are constant talks in the news about an impending recession. Yes, there are signs, such as the recent yield curve inversion, the market corrections in 2018 & 2019, etc. Recessions happen every 10 years or so, are we overdue for one?

The first part is to predict GDP growth, because NBER defines recessions as ['significant decline in economic activity'](https://www.nber.org/cycles.html), not 2 consequtive quarters of deline in real GDP. This means GDP can be growing, albeit at a slow pace, and the economy is still considered in recession.

But due to limitations of the VAR model employed in part 1, it is necessary to employ an alternative method, to ______.

## Part 1

#### Feature Selection

While there are techniques and algorithms to systematically select variables, this is outside the scope of this analysis. Instead, we will rely on domain knowledge. The following factors are considered when selecting input variables:
* Having sufficient data, as recessions are rare occurances.
* Having forward predictive power, which means inflection changes should occur before occur simultaneously as inflection changes in GDP.
* Represent various aspects and mechanisms of the economy, such as a risk free benchmark, or inflation.
* Avoid composite models or indices, as the goal is to construct our own model.

For the GDP decision variable:
* [**Real Gross Domestic Product**](https://fred.stlouisfed.org/series/GDPC1)- Inflation adjusted value of goods and services produced in a year.

For equities, the following variables are considered:
* [S&P 500](https://finance.yahoo.com/quote/%5EGSPC?p=^GSPC)- Weighted stock index of 500 companies listed on US exchanges with the largest market cap.
* [S&P 500 Cyclically Adjusted PE Ratio](https://www.quandl.com/data/MULTPL/SHILLER_PE_RATIO_MONTH-Shiller-PE-Ratio-by-Month) [(info)](https://www.investopedia.com/terms/c/cape-ratio.asp)- Normalizes PE ratio fluctuations over 10 year inflation-adjusted earnings. This indicator is used to gauge whether the equities market is over or under-valued.
* [**Nonfinancial Corporate Debt as Percentage of Equity**](https://fred.stlouisfed.org/series/NCBCMDPMVCE)- Contains companies not in the S&P 500. Although this indicator excludes [% of GDP](), valuations of financial and noncorporate companies 

For bonds, the following variables are considered: 
* [10 Year Constant Maturity Minus 3 Month Treasuries Yield Spread](https://fred.stlouisfed.org/series/T10Y3M)- Federal Reserve's main methodology. Financial institutions borrow at low rates short term, to lend at high rates long term. Long term treasury yield tends to be stable, but short term yields are more volatile, and subject to [Federal Reserve interventions]((https://fredblog.stlouisfed.org/2017/02/lets-do-the-twist/)). The data only goes back to 1982, which is not enough to train this model.
* [**10 Year Constant Maturity Minus 3 Month Treasuries Secondary Market Yield Spread**](https://fred.stlouisfed.org/series/TB3MS)- Makes up for the lack of data in the primary market. Primary and secondary market spreads are very close today, due to increased efficiency from electronic trading. Historically, the spread has been higher.
* [10 Year Treasuries Constant Maturity Rate](https://fred.stlouisfed.org/series/DGS10)- Long term treasuries usually reflect investor sentiment regarding long term economic growth. However, this metric alone does not provide evidence of short term liquidity.
* 3 Months Outstanding Repo [(info)](http://law.emory.edu/ecgar/content/volume-5/issue-2/essays/repo-recession-financial-regulation.html)- Before the Great Recession, investment banks used short term repo to inject liquidity to stay afloat. Uptick in short term repo may indicate credit crunch. FRED only has records of contracts with itself as a participant, leaving out the majority of transactions. Data on market transactions were difficult to find.

For inflation, the following indicators are considered:
* [**Consumer Price Index for Urban Consumers**](https://fred.stlouisfed.org/series/CPIAUCSL) [(info)](https://www.bls.gov/opub/btn/volume-3/why-does-bls-provide-both-the-cpi-w-and-cpi-u.htm)- Price inflation covering 88% of Americans, calculated from expenditures. Uses a survey to calculate the basket of goods and services. Basket is updated every 2 years. Selected due to difficulty quantifying intangible traits such as quality.
* [Chained Consumper Price Index for Urban Consumers](https://fred.stlouisfed.org/series/SUUR0000SA0) [(info)](https://www.brookings.edu/blog/up-front/2017/12/07/the-hutchins-center-explains-the-chained-cpi/)- Lacking long term data. Similar to CPI-U, but considers substitution purchases, and weights the changes every month.

For employment, the following indicators are considered:
* [Labor Participation Rate](https://fred.stlouisfed.org/series/CIVPART)- Percent of population over 16 actively seeking or engaged in employment. Too macro driven, such as women participation in the workforce, or retirement of Baby Boomers.
* [**Total Nonfarm Payroll**](https://fred.stlouisfed.org/series/PAYEMS)- Size of the labor force, accounting for 80% of workers who contribute to GDP, excluding propritors, unpaid volunteers, or farm workers.
* [Wage Growth](https://fred.stlouisfed.org/series/CES0500000003)- When labor demand outstrips supply, wages increase, and vise versa. Lack of long term data.

Lastly, the following indicators are not placed in any categories above, but are included because of their predictive power.
* [**Real Private Gross Investment**](https://fred.stlouisfed.org/series/GPDIC1)- Investment represents expenditure on capital goods and residential properties. Provides an indicator for future productivity and GDP growth. Also a strong sign of economic recovery.

The following indicators were considered, but ultimately dismissed.
* [Purchaing Managers' Index](https://www.instituteforsupplymanagement.org/ismreport/mfgrob.cfm?SSO=1)- Locked behind paywall. A survey of supply chain professionals in 19 primary industries.
* [Composite Index of Leading Indicators](https://www.conference-board.org/data/bcicountry.cfm?cid=1) [(info)](https://www.investopedia.com/terms/c/cili.asp)- Locked behind paywall. A composite index of 10 indicators.
* [Consumer Confidence Index](https://www.conference-board.org/data/consumerconfidence.cfm)- Locked behind paywall. A survey of consumer purchases and sentiments.
* [VIX Volativity Index](https://fred.stlouisfed.org/series/VIXCLS)- Describes the volatility, but not the direction, of stocks. Also does not describe economic strength.
* [Effective Federal Funds Rate](https://fred.stlouisfed.org/series/FEDFUNDS)- The most inorganic bond indicator, as the FED has direct control of this rate, which means it is unfiltered by market forces. Secondly, different FED chairs have different doctrines. The FED was fighting inflation in the 70s, deregulation in the 80s, inflation targeting in the 90s & 2000s, then quantitative easing in the 2010s.
* [Personal Savings Rate](https://fred.stlouisfed.org/series/PSAVERT)- Too macro driven, such as women entering workforce, or retirement of Baby Boomers.
* Incremental Capital Outputs Ratio- Hard to find on the web. Calculates how much additional capital investment is needed to create growth.

## Data Preparation

First, import the necessary packages.

In [2]:
import fredapi
import requests
import json
import numpy as np
import pandas as pd
import scipy as sp
from functools import reduce
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sb
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import acf
from copy import deepcopy
from dateutil.relativedelta import relativedelta

#### Import FRED Data

To import FRED data, we need an API key. API key can be requested [here](https://research.stlouisfed.org/docs/api/fred/).

Data returned by fredapi module are in Series format. We need to convert them to DataFrame format.

We will also rename the columns so the data become easier to manipulate.

In [3]:
f = fredapi.Fred(api_key='8b91217446b6307d20cb5e4fcfba70eb') # remove API when done

# monthly data.
tbill_3m = f.get_series('TB3MS').to_frame().reset_index().rename(columns={'index':'date', 0:'tbill_3m'})
tbill_10y = f.get_series('GS10').to_frame().reset_index().rename(columns={'index':'date', 0:'tbill_10y'})
cpiu = f.get_series('CPIAUCSL').to_frame().reset_index().rename(columns={'index':'date', 0:'cpiu'})
payroll = f.get_series('PAYEMS').to_frame().reset_index().rename(columns={'index':'date', 0:'payroll'})
# quarterly data
dtoe = f.get_series('NCBCMDPMVCE').to_frame().reset_index().rename(columns={'index':'date', 0:'dtoe'})
pinvest = f.get_series('GPDIC1').to_frame().reset_index().rename(columns={'index':'date', 0:'pinvest'})
gdp = f.get_series('GDPC1').to_frame().reset_index().rename(columns={'index':'date', 0:'gdp'})

#### Import Quandl Data

In [4]:
quandl_api_key = '8ufKe7Y2JMsYPU3CGN7m'
sp500 = pd.DataFrame.from_dict(requests.get('https://www.quandl.com/api/v3/datasets/MULTPL/SP500_REAL_PRICE_MONTH.json?api_key={}'.format('8ufKe7Y2JMsYPU3CGN7m')).json()['dataset']['data'])
sp500_cape = pd.DataFrame.from_dict(requests.get('https://www.quandl.com/api/v3/datasets/MULTPL/SHILLER_PE_RATIO_MONTH.json?api_key={}'.format('8ufKe7Y2JMsYPU3CGN7m')).json()['dataset']['data'])

In [16]:
# reverse order
# index date
# rename columns
# sp500 = sp500.rename(columns={0:'date', 1:'sp500'})
# sp500['date'] = pd.to_datetime(sp500['date'])
# sp500 = sp500[::-1].set_index('date')
# sp500_cape = sp500_cape[::-1].rename(columns={0:'date', 1:'sp500_cape'})
sp500_cape = sp500_cape[::-1].rename(columns={0:'date', 1:'sp500'}).set_index('date')
sp500_cape.index = sp500_cape.index.astype('datetime64')

/home/chrx/miniconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:346: FutureWarning: Passing in 'datetime64' dtype with no precision is deprecated
and will raise in a future version. Please pass in
'datetime64[ns]' instead.
  data, copy=copy, name=name, dtype=dtype, **kwargs


In [17]:
print(sp500_cape.info())
sp500_cape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1787 entries, 1871-02-01 to 2019-11-01
Data columns (total 1 columns):
sp500    1787 non-null float64
dtypes: float64(1)
memory usage: 27.9 KB
None


,sp500
date,
1871-02-01,10.92
1871-03-01,11.19
1871-04-01,12.05
1871-05-01,12.59
1871-06-01,12.59
...,...
2019-07-01,29.99
2019-08-01,28.65
2019-09-01,28.64


In [7]:
print(sp500.info())
sp500

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1788 entries, 1871-01-01 to 2019-11-01
Data columns (total 1 columns):
sp500    1788 non-null float64
dtypes: float64(1)
memory usage: 27.9 KB
None


,sp500
date,
1871-01-01,4.44
1871-02-01,4.50
1871-03-01,4.61
1871-04-01,4.74
1871-05-01,4.86
...,...
2019-07-01,2996.11
2019-08-01,2883.98
2019-09-01,2906.27


#### Setting Start Date

Note that variables have different startig dates, which we need to standardize; doing so also removes any null values.

In [ ]:
print(tbill_3m.head(1))
print(tbill_10y.head(1))
print(cpiu.head(1))

For consistency, we will set the starting date at 1953/4/1.

In [ ]:
start_date = pd.to_datetime('1953, 4')
tbill_3m = tbill_3m[tbill_3m.iloc[:,0] >= start_date]
tbill_10y = tbill_10y[tbill_10y.iloc[:,0] >= start_date]
cpiu = cpiu[cpiu.iloc[:,0] >= start_date]
payroll = payroll[payroll.iloc[:,0] >= start_date]
pinvest = pinvest[pinvest.iloc[:,0] >= start_date]
dtoe = dtoe[dtoe.iloc[:,0] >= start_date]
gdp = gdp[gdp.iloc[:,0] >= start_date]
sp500 = sp500[sp500.iloc[:,0] >= start_date]
sp500_cape = sp_500[sp_500.iloc[:,0] >= start_date]

#### Deriving Correct Treasury Spread

FRED have a graph, not series of data on [10 year - 3 month spread](https://fred.stlouisfed.org/graph/?g=oGg). We have to creat our own.

In [ ]:
tbill = [tbill_10y, tbill_3m]
tbill = reduce(lambda left,right: pd.merge(left,right,on='date'), tbill)
tbill = tbill.set_index('date')
tbill['tbill_10y'] = tbill.loc[:,'tbill_10y'] - tbill.loc[:,'tbill_3m']
tbill = tbill.drop('tbill_3m', axis=1)
tbill = tbill.rename(columns={'tbill_10y':'tbill'})

#### Converting Quarterly to Monthly Data and Null Value Interpolation

To convert quarterly to monthly data, we will call the "resample" method.

In [ ]:
dtoe = dtoe.set_index('date').resample('MS').asfreq()
pinvest = pinvest.set_index('date').resample('MS').asfreq()
gdp = gdp.set_index('date').resample('MS').asfreq()

In [ ]:
gdp.head(2)

Newly resampled months have null values, we will use [cublic spline interpolation](https://columbiaeconomics.com/2010/01/20/how-economists-convert-quarterly-data-into-monthly-cubic-spline-interpolation/), a technique commonly used by economists, to fill in the missing values. Cubic spline has better smoothing over quadratic interpolation.

Interpolation converts data to Series. We need to convert back to DataFrame type.

In [ ]:
gdp = gdp['gdp'].interpolate(method='spline', order=3).to_frame().reset_index()
dtoe = dtoe['dtoe'].interpolate(method='spline', order=3).to_frame().reset_index()
pinvest = pinvest['pinvest'].interpolate(method='spline', order=3).to_frame().reset_index()

### Preliminary Analysis

Join all variables into the the same dataframe for easy manipulation.

In [ ]:
data = [tbill, cpiu, payroll, dtoe, pinvest, gdp]
data = reduce(lambda left,right: pd.merge(left,right,on='date'), data)
data = data.set_index('date')

data.head(2)

Verify there are no remainng null values.

In [ ]:
data.isnull().sum()

There are 793 months of data in the dataset.

In [ ]:
data.describe()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, dpi=120, figsize=(10,6))
for col, ax in enumerate(axes.flatten()):
    ax.plot(data[data.columns[col]], color='red', linewidth=1)
    ax.set_title(data.columns[col])
    ax.tick_params(labelsize=8)
    
plt.tight_layout()

#### Check for Correlation

In [ ]:
plt.figure(figsize = (10,5))
ax = sb.heatmap(data.corr(), xticklabels=data.columns.values, yticklabels=data.columns.values, annot=True, annot_kws={'size':12})
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

CPIU, PAYROLL, PINVEST, and GDP are highly correlated. This could create problems of multicollinearity. Multicollinearity reduces the statistical significance of the variables, making it difficult to pinpoint which variable is responsible for changes in GDP.

Going back to the criteria set at the beginning, CPIU and PAYROLL will be removed. While PINVEST have larger variance, which may obsecure the signal, PINVEST is kept because it has the earliest inflection point.

In [ ]:
data = data.drop(['cpiu', 'payroll'], axis=1)

#### Model Selection

For dealing with multiple variables, a vector autoregression (VAR) model is right for the job.

VAR is a stochastic (random) process that captures linear relationship between all variables. VAR assumes all variables are dependent on each other, and that change in 1 variable will induce change in another. This trait is called Granger causality. We can verify if we should reject this hypothesis.

#### Test for Autocorrelation

GDP is highly autocorrelated. While this feature can allow a generalization of GDP trend (2% year-over-year growth), we are interested in the short term fluctuations. Having high autocorrelation does not help to delect these fluctuations.

** Expalin full vs. partial autocorrelation **.

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 6))
ax[0] = plot_acf(data.gdp, ax=ax[0])
ax[1] = plot_pacf(data.gdp, ax=ax[1])

#### Test for Stationarity

It is also important for all data to be stationary, meaning the mean and variance do not change overtime. The primary feature to eliminate is trend, for example: GDP increases steadily over the years, the mean increases as time increases, and so does the variance. At the current trend, as time approaches infinity, so does the mean and variance. Many data models do not deal with changing mean and variance very well.

In [ ]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        
    def ADF_Stationarity_Test(self, timeseries):
        adfTest = adfuller(timeseries, autolag='AIC')
        self.pValue = adfTest[1]
        
        if (self.pValue < self.SignificanceLevel):
            self.isStationary = True
        else:
            self.isStationary = False
        
        dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags','# Observations'])
        dfResults['Critical Value 5%'] = adfTest[4]['5%']

        print('Augmented Dickey-Fuller Test Results:')
        print(dfResults)

In [ ]:
sTest = StationarityTests()

for var in data:
    sTest.ADF_Stationarity_Test(data[var])
    print('Is {} series stationary? {}'.format(var, sTest.isStationary))
    print('\n')

#### Data Transformation

To make our data stationary, we will take the first difference of month(t):
y_t - y_(t-1)

In [ ]:
data_trans = data.diff().dropna()

#### Splitting Data Into Trainng and Testing Sets

In [ ]:
test_periods = 12
train, test = data[0:-test_periods], data[-test_periods:]
train_trans = data_trans[0:-test_periods]
train_trans.tail()

Checking for autocorrelation, much have been eliminated, but some degrees 

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 6))
ax[0] = plot_acf(train_trans.gdp, ax=ax[0])
ax[1] = plot_pacf(train_trans.gdp, ax=ax[1])

In [ ]:
for var in train:
    sTest.ADF_Stationarity_Test(train_trans[var])
    print("Is {} series stationary? {}".format(var, sTest.isStationary))
    print('\n')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, dpi=120, figsize=(10,6))
for col, ax in enumerate(axes.flatten()):
    ax.plot(train_trans[train_trans.columns[col]], color='red', linewidth=1)
    ax.set_title(train_trans.columns[col])
    ax.tick_params(labelsize=8)
    
plt.tight_layout()

In [ ]:
plt.figure(figsize = (10,5))
ax = sb.heatmap(train_trans.corr(), xticklabels=train_trans.columns.values, yticklabels=train_trans.columns.values, annot=True, annot_kws={'size':12})
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

#### Lag Order Selection

VAR uses 4 metrics for lag order selection: 

* Aikaike Information Criterion (AIC)
* Baysian Information Criteron (BIC) 
* Final Prediction Error (FPE)
* Hannan-Quinn Information Criterion (HQIC)

Not getting into the specific mechanics, these four metrics return the error value. We want the lag orders with the lowest error value. 

https://stats.stackexchange.com/questions/246886/selecting-lag-order-for-var-and-vecm

AIC has the tendency to chooose large lag order, while BIC has the tendency to choose small lag order. For that reason, the average of AIC and BIC will be used. The average lag order is 14.5, so we will select 15.

In [ ]:
train_model = VAR(train_trans)
train_model.select_order(maxlags=36).summary()

#### Granger Causality & Cointegration Test

Granger causality test assumes there is no relationship between the lag value of x(row) and present value of y(column), if resultant p value is greater than 0.05.

The heat map below indicates *pinvest* has no causality relations, but the p-value is not too far from the threshold. While this raises red flag regarding our model, implying different variable and transformation selection methods should be employed, we can verify the merit of this interpretation through cointegration test.

In [ ]:
lag_order = 21

def grangerstest(data, variables, test='ssr_chi2test'):
    res = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)

    for c in data.columns:
        for r in res.index:
            granger_res = grangercausalitytests(data[[r, c]], maxlag=lag_order, verbose=False)
            p_values = [round(granger_res[i+1][0][test][1],4) for i in range(lag_order)]

            res.loc[r, c] = min(p_values)
    res.columns = [var + '_x' for var in variables]
    res.index = [var + '_y' for var in variables]
    return res

gct = grangerstest(train_trans, variables=train_trans.columns)

In [ ]:
plt.figure(figsize = (10,5))
ax = sb.heatmap(gct, xticklabels=gct.columns.values, yticklabels=gct.index.values, annot=True, annot_kws={'size':12})
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

plt.show()

Cointegration is simply that by lagging one variable against another (order of integration or lag order), the distance between variables is stationary throughout time. If two variables are cointegrated, there must be granger causality (even if a weak one, or one way causality).

Johansen test combines two series together, than tests whether this combined series has less order of integration than a single variable on its own. If so, then the variables are cointegrated.

Here we pass in 3 variables:
1. data,
2. 1 for stochastic nature of the data,
3. number of lagged periods.

While there are many literature on whether the economy is [deterministic or stochastic](https://fraser.stlouisfed.org/files/docs/historical/frbsf/frbsf_let/frbsf_let_19930212.pdf), with proponents of a deterministic model arguing GDP hinders on the long term supply factors, such as capital, labor, and technology, and that growth rate should stay the same overtime; while proponents of a stochastic model would point to "permanent" changes to the growth rate, positive or negative, such as sudden sharp slope increase in supply factors since the industrial revolution. Under a deterministic model, proponents of stochastic model would say, people would still be living in the 17th century.

In [ ]:
def cointegration_test(data, det_order, lag_order, significance=0.05):
    out = coint_johansen(data, det_order, lag_order)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-significance)]]

    def adjust(val, length=6): return str(val).ljust(length)
    print('Name   ::  Test Stat  >  C(95%)    => Signif  \n', '--'*20)
    for col, trace, cvt in zip(data.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace, 2), 9), '> ', adjust(cvt, 8), ' => ', trace > cvt)
        
cointegration_test(train_trans, det_order=-1, lag_order=8)

Testing lag order of 15, *gdp* fails the cointegration test, so we will decrease the lag order until all four variables are cointegrated. We do not want the lag order to be too small, as small lag orders tend to be autocorrelated.

Below is a modified function to find the highest lag order that satisfies the cointegration test.

In [ ]:
def cointegration_test_lag_order_detection(data, det_order, lag_order, significance=0.05):
    out = coint_johansen(data, det_order, lag_order)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-significance)]]
    res = traces > cvts
    return res

for i in reversed(range(1, 15)):
    print('lag order:', i, cointegration_test_lag_order_detection(train_trans, det_order=-1, lag_order=i))

#### VAR Model

In [ ]:
lag_order = 8
train_model_fitted = train_model.fit(lag_order)
train_model_fitted.summary()

#### Check for Residual Correlation with Durbin Watson Test

If our transformation and lag order selection is done correctly, there should be little to no correlation between the residual errors. The existance of residual correlation implies unexplained pattern.

Result values can range beween 0 to 4, with 0-2 indicating a positive correlation, and 2-4 indicating negative correlation. It is generally acceptable for values to be between 1.5 to 2.5.

In [ ]:
for col, val in zip(train_trans.columns, durbin_watson(train_model_fitted.resid)):
    print((col), ':', round(val, 2))

#### Forecasting GDP

In [ ]:
train_forecast_input = train_trans.values[-lag_order:]
train_forecast_input

In [ ]:
train_forecast = pd.DataFrame(train_model_fitted.forecast(train_forecast_input, test_periods), 
                        index=data_trans.index[-test_periods:], 
                        columns=data_trans.columns)

train_forecast

#### Inverting Transformation

In [ ]:
train_forecast_reverse = train_forecast.cumsum() + train.iloc[-1, :]
train_forecast_reverse

In [ ]:
fig, axes = plt.subplots(nrows=int(len(train_forecast_reverse.columns)/2), ncols=2, dpi=120, figsize=(10,6))
for i, (col,ax) in enumerate(zip(train_forecast_reverse.columns, axes.flatten())):
    train_forecast_reverse[col].plot(color='red', label='prediction', legend=True, ax=ax).autoscale(axis='x',tight=True)
    test[col].plot(color='blue', label='actual', legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

For some reason this model was not able 

* MAPE- 
* ME- 
* MAE- 
* MPE- 
* RMSE- 
* Minmax- 

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual) / np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual) / actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins / maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

In [ ]:
def print_forecast_accuracy(forecast, actual):
    for c in forecast:
        print('Forecast Accuracy of {}'.format(c))
        accuracy_prod = forecast_accuracy(forecast[c].values, actual[c])
        for k, v in accuracy_prod.items():
            print(k, ': ', round(v,4))
        print('\n')

print_forecast_accuracy(forecast=train_forecast_reverse, actual=test)

In [ ]:
# model = VAR(data_trans)
# model_fitted = train_model.fit(lag_order)

# forecast = pd.DataFrame(train_model_fitted.forecast(data_trans.values[-lag_order:], test_periods), 
#                         index=data_trans.index[-test_periods:],
#                         columns=data_trans.columns)
# forecast_reverse = forecast.cumsum() + data.iloc[-1, :]

# forecast_reverse.index = pd.date_range(pd.to_datetime('2019, 05, 1'), pd.to_datetime('2020, 04, 1'), freq='MS')
# forecast

In [ ]:
# fig, axes = plt.subplots(nrows=int(len(forecast_reverse.columns)/2), ncols=2, dpi=120, figsize=(10,6))
# for i, (col,ax) in enumerate(zip(forecast_reverse.columns, axes.flatten())):
#     test[col].plot(color='blue', label='actual', legend=True, ax=ax)
#     forecast_reverse[col].plot(color='red', label='prediction', legend=True, ax=ax).autoscale(axis='x',tight=True);
#     ax.set_title(col + ": Forecast")
#     ax.xaxis.set_ticks_position('none')
#     ax.yaxis.set_ticks_position('none')
#     ax.spines["top"].set_alpha(0)
#     ax.tick_params(labelsize=6)

# plt.tight_layout();

#### Consolidate Formula

In [ ]:
def var_combined(data_input, lag_order, start_date, test_periods):
    '''return start date'''
    start_date = pd.to_datetime(start_date)
    
    '''dates for graphing'''
    data_baseline = data_input.loc[(data_input.index >= start_date - relativedelta(months=2 * test_periods)) & 
                                    (data_input.index <= start_date + relativedelta(months=test_periods))]
    
    '''take first first difference'''
    data_diff = data_input.diff().dropna()
    data_diff = data_diff.loc[data_diff.index <= start_date]

    '''VAR model'''
    model = VAR(data_diff)
    model_fitted = train_model.fit(lag_order)
    forecast_diff = pd.DataFrame(
        model_fitted.forecast(data_diff.values[-lag_order:], test_periods), 
        index=data_diff.index[-test_periods:], columns=data_diff.columns
    )
    
    '''invert transformation'''
    '''boolean index returns dataframe, but needs to be series'''
    forecast_reverse = forecast_diff.cumsum() + data_input.iloc[data_input.index == start_date].squeeze()
#     forecast_reverse.index = pd.date_range(pd.to_datetime('2019, 05, 1'), pd.to_datetime('2020, 04, 1'), freq='MS')
    forecast_reverse.index = pd.date_range(start_date + relativedelta(months=1), start_date + 
                                           relativedelta(months=test_periods), freq='MS')
#     print(forecast_reverse.tail(1))
#     print(data_diff.tail(1))
#     print(data_input.tail(1))

    '''display graphs'''
    fig, axes = plt.subplots(nrows=int(len(forecast_reverse.columns)/2), ncols=2, dpi=120, figsize=(7,4))
    for i, (col,ax) in enumerate(zip(forecast_reverse.columns, axes.flatten())):
        data_baseline[col].plot(color='blue', label='actual', legend=True, ax=ax)
        forecast_reverse[col].plot(color='red', label='prediction', legend=True, ax=ax).autoscale(axis='x',tight=True);
        ax.set_title(col + ": Forecast")
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        ax.spines["top"].set_alpha(0)
        ax.tick_params(labelsize=6)
        plt.tight_layout();

In [ ]:
var_combined(data, lag_order=8, start_date='2018, 4', test_periods=36)

## Findings

## Limitations

#### Feature Selection

This predictor does not pinpoint the cause of a recession, but makes a general comment about the state of the economy. The underlying assumption is that these inputs are efficient enough to reflect long term market conditions, but inefficient enough to price in the short term fluctuations in GDP.

When trying to quantify debt as percentage of equity, it turned out that noncoporate entities 

The model only accepts data input from the determined lag order, meaning only 15 months of data was used.  The limited dataset does not include a recession, meaning the w

#### VAR Model

## Part 2

#### Data Preparation

Let's set up recession decision variable. Difficulty joining series, so had to convert to list, append, then reconvert to series.

In [ ]:
# possible to create lambda variables?
r_1953 = pd.Series(pd.date_range(pd.to_datetime('1953, 3, 1'), pd.to_datetime('1954, 5, 1'))).tolist()
r_1957 = pd.Series(pd.date_range(pd.to_datetime('1957, 8, 1'), pd.to_datetime('1958, 4, 1'))).tolist()
r_1960 = pd.Series(pd.date_range(pd.to_datetime('1960, 4, 1'), pd.to_datetime('1961, 2, 1'))).tolist()
r_1962 = pd.Series(pd.date_range(pd.to_datetime('1969, 12, 1'), pd.to_datetime('1970, 11, 1'))).tolist()
r_1973 = pd.Series(pd.date_range(pd.to_datetime('1973, 11, 1'), pd.to_datetime('1975, 3, 1'))).tolist()
r_1980 = pd.Series(pd.date_range(pd.to_datetime('1980, 1, 1'), pd.to_datetime('1980, 7, 1'))).tolist()
r_1981 = pd.Series(pd.date_range(pd.to_datetime('1981, 7, 1'), pd.to_datetime('1982, 11, 1'))).tolist()
r_1990 = pd.Series(pd.date_range(pd.to_datetime('1990, 7, 1'), pd.to_datetime('1991, 3, 1'))).tolist()
r_2001 = pd.Series(pd.date_range(pd.to_datetime('2001, 3, 1'), pd.to_datetime('2001, 11, 1'))).tolist()
r_2007 = pd.Series(pd.date_range(pd.to_datetime('2007, 12, 1'), pd.to_datetime('2009, 6, 1'))).tolist()

print('Original series length:', len(r_1953))

recession_months = pd.Series(r_1953 + r_1957 + r_1960 + r_1962 + r_1973 + r_1980 + r_1981 + r_1990 + r_2001 + r_2007)

# Create a DataFrame for recession
start_date = pd.to_datetime('1953, 4')
end_date = pd.to_datetime('today')
recessions = pd.date_range(start_date, end_date, freq='MS')
recessions = recessions.to_frame().reset_index()

# Mark recession months as 1, non-recession months as 0
recessions.iloc[:,1] = 0
recessions.loc[recessions.iloc[:,0].isin(recession_months), 0] = 1
recessions.rename(columns={'index':'date', 0:'recession'}, inplace=True)


print('Verify series was appended:', len(recession_months))

recessions.iloc[15:20]

## Closing Thoughts

Special thanks to Mortada Mehyar for creating [fredapi](https://github.com/mortada/fredapi). This saved some time from having to deal with JSON format.